# Scraping Offical German Government Website
- germany has different federal states and each federal state has kreise and each kreis has an amtsgericht in which the Zwangsversteigerungen take place
- each amtsgericht uploads the zwangsversteigerungen to https://www.zvg-portal.de/index.php?button=Suchen&all=1, this offers only a generale overview
- there are different of information in our data, the general description, the exposee, gutachten and amtliche bekanntmachung and fotos
- we only want to focus on the general description, amtliche bekanntmachung and exposee when scraping, we will safe the gutachten as external link and the fotos, most of the time the fotos arent actually really helpful, because its only pictures from outside the apartment or house and the grundrisse are also meh, but we will safe this as external links, if we want to use this data later
- the gutachten has also some interesting informations, but from the gutachten we get an information overload, what we could try is to feat 
- value they delivier and test especially for Berlin if we get better predictions in our model with these informations (looking into regression by picture or translating a picture into text) 
- then there are are websites which seem to scrape the government website and display it with less information but in a more stylized way like (https://versteigerungspool.de/, https://www.zwangsversteigerung.de/ or https://www.zvg-online.net/), we dont want to scrape such sites, because they offer us no additional value, except for easier scraping in some parts

- https://www.zvg-portal.de/ (official website of the german government where foreclosure auctions can be found for selected regional courts) 
  - (Effort: uniform general description, more detailed information in unstructured PDFs, which would need to be extracted)
  - https://www.zvg-portal.de/index.php?button=Suchen&all=1
  - alle 16 Bundesländer haben einen kürzel auf dieser Website, diese Kürzel kann ich entweder aus https://www.zvg-portal.de/index.php?button=Termine%20suchen entnehmen oder als constante speichern, bevorzugter Weise von der Webseite entnehmen
  - Alerting, wenn für ein Bundesland keine neuen Daten kamen
  - pandas df in sqllite db speichern anstatt in csv
  - PDF Daten müssen vorher schon entweder als string mit in die DB gegeben werden oder am besten schon gleich processen, damit ich nicht zu viele Daten speichere
  - Gutachten, Amtliche Bekanntmachung, Exposee erstmal als external Links speichern
  - How to scrape:
    - go to https://www.zvg-portal.de/index.php?button=Termine%20suchen form -> iterate through table first tr td should contain Land
    - if it contains Land then do: go to next tr.td.select inside select are options take each option value attribute except for 0
    - after having all values do a post request for each saved value https://www.zvg-portal.de/index.php?button=Suchen&all=1 with body ger_name=--+Alle+Amtsgerichte+--&order_by=2&land_abk=value&ger_id=0&az1=&az2=&az3=&az4=&art=&obj=&str=&hnr=&plz=&ort=&ortsteil=&vtermin=&btermin=
      - inside form the second table go through all trs if tr contains three tds, go to the second td, get the a element link 
      - open the link
      - get table first tr contains in first td aktenzeichen and in second td letzte Aktualisierung
      - after first tr go through all other trs if first td in a tr equals to key[: -1] in keywords (keywords={"Art der Versteigerung", "Grundbuch", "Objekt/Lage", "Beschreibung", "Verkehrswert in €", "Termin", "Ort der Versteigerung", "amtliche Bekanntmachung", "Exposee", "Gutachten"}) then safe the value if value contains a then safe the link
      - if key == "Foto" then safe link in list
      - after finishing iteration safe foto list in fotos and append everything saved as new row to an sqlite db 

In [1]:
from sqlalchemy import Column, String, Text, Integer, create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel, Field, model_validator
from typing import Optional
from datetime import datetime
import locale
import logging
import os

locale.setlocale(locale.LC_TIME, 'de_DE')
os.makedirs("logs", exist_ok=True)
logging.basicConfig(
    filename=f"logs/foreclosure_scraper_{int(datetime.now().timestamp())}.log",
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [14]:
Base = declarative_base()

class ForeclosureCaseSchema(Base):
    __tablename__ = 'foreclosure_data'

    link = Column(String, nullable=False, primary_key=True)
    bundesland_code = Column(String, nullable=False)
    aktenzeichen = Column(String, nullable=False)
    letzte_aktualisierung = Column(String)
    art_der_versteigerung = Column(String)
    grundbuch = Column(String)
    objekt_lage = Column(String)
    beschreibung = Column(Text)
    verkehrswert = Column(String)
    termin = Column(Integer)
    ort_der_versteigerung = Column(String)
    amtliche_bekanntmachung = Column(String)
    exposee = Column(String)
    gutachten = Column(String)
    foto = Column(Text)

In [15]:
engine = create_engine('sqlite:///foreclosures.db')
Session = sessionmaker(bind=engine)
session = Session()
Base.metadata.create_all(engine)

#### Federal state codes
- TODO: Description

In [16]:
url = "https://www.zvg-portal.de/index.php?button=Termine%20suchen"
response = requests.get(url)
search_soup = BeautifulSoup(response.text, 'html.parser')

land_select = None
for tr in search_soup.find_all('tr'):
    if "Land" in tr.text:
        land_select = tr.find_next('select')
        break

if not land_select:
    print("Land select field not found.")
    exit()

land_codes = [option['value'] for option in land_select.find_all('option') if option['value'] != "0"]

#### Auctions
- TODO: Description

In [5]:
class ForeclosureCaseModel(BaseModel):
    link: str
    aktenzeichen: str
    letzte_aktualisierung: int
    art_der_versteigerung: Optional[str] = None
    grundbuch: Optional[str] = None
    objekt_lage: Optional[str] = None
    beschreibung: Optional[str] = None
    verkehrswert: Optional[str] = Field(None, alias='verkehrswert_in_€')
    termin: Optional[int] = None
    ort_der_versteigerung: Optional[str] = None
    amtliche_bekanntmachung: Optional[str] = None
    exposee: Optional[str] = None
    gutachten: Optional[str] = None
    foto: Optional[str] = None

    class Config:
        populate_by_name = True

    @model_validator(mode='before')
    def convert_letzte_aktualisierung(cls, values):
        letzte_aktualisierung_str = values.get('letzte_aktualisierung')
        if letzte_aktualisierung_str and isinstance(letzte_aktualisierung_str, str):
            date_str = letzte_aktualisierung_str[1:-1].split(": ", 1)[1]
            date_obj = datetime.strptime(date_str, '%d-%m-%Y %H:%M')
            values['letzte_aktualisierung'] = int(date_obj.timestamp())
        return values

    @model_validator(mode='before')
    def convert_termin(cls, values):
        termin_str = values.get('termin')
        if termin_str and isinstance(termin_str, str):
            date_obj = datetime.strptime(termin_str, '%A, %d. %B %Y, %H:%M Uhr')
            values['termin'] = int(date_obj.timestamp())
        return values

    @model_validator(mode='before')
    def convert_fotos(cls, values):
        fotos_list = values.get('foto')
        if isinstance(fotos_list, list):
            values['foto'] = ','.join(fotos_list)
        return values

In [6]:
def get_foreclosure_case_data(link: str) -> ForeclosureCaseModel:
    response = requests.get(link, headers={"Referer": "https://www.zvg-portal.de/index.php?button=Suchen"})
    if not str(response.status_code).startswith("2"):
        raise Exception(f"Request for {link} failed!")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    details_table = soup.find_all('table')[0]
    rows = details_table.find_all('tr')
    aktenzeichen = rows[0].find_all('td')[0].text.strip()
    letzte_aktualisierung = rows[0].find_all('td')[1].text.strip()
    
    foreclosure_case = {
        "link": link,
        "aktenzeichen": aktenzeichen,
        "letzte_aktualisierung": letzte_aktualisierung
    }
    
    for row in rows[1:]:
        key = row.find_all('td')[0].text.strip()[:-1].lower().replace(" ", "_").replace("/", "_")
        if key in ForeclosureCaseModel.model_fields or key == "verkehrswert_in_€":
            second_cell = row.find_all('td')[1]
            anchor_tags = second_cell.find_all('a')
            img_tags = second_cell.find_all('img')
            if len(anchor_tags) == 1 and len(img_tags) == 1:
                if key == "foto":
                    foreclosure_case.setdefault('foto', []).append(anchor_tags[0]['href'].strip())
                    continue
                value = anchor_tags[0]['href'].strip()
            else:
                value = second_cell.text.strip()

            foreclosure_case[key] = value
    
    return ForeclosureCaseModel(**foreclosure_case)

In [7]:
for land_code in land_codes:
    post_url = "https://www.zvg-portal.de/index.php?button=Suchen&all=1"
    post_data = {
        'ger_name': '--+Alle+Amtsgerichte+--',
        'order_by': '2',
        'land_abk': land_code,
        'ger_id': '0'
    }
    
    post_response = requests.post(post_url, data=post_data)
    if not str(post_response.status_code).startswith("2"):
        logging.error(f"Request for {land_code} failed! Status Code: {post_response.status_code}")
        continue
    
    print(f"Working on {land_code}")
    land_soup = BeautifulSoup(post_response.text, 'html.parser')
    result_tables = land_soup.find_all('table')
    if not result_tables or len(result_tables) < 2:
        logging.info(f"No foreclosure cases for {land_code}!")
        continue
    
    result_table = result_tables[1]
    for tr in result_table.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 3 and tds[0].text.strip() == "Aktenzeichen":
            if not tds[1].find('a'):
                logging.info(f"Foreclosure Case already expired!")
                continue
            
            link = tds[1].find('a')['href']
            case_url = f"https://www.zvg-portal.de/{link}"
            
            if session.query(ForeclosureCaseSchema).filter_by(link=case_url).first():
                logging.warning(f"Foreclosure Case {case_url} already exists in the database!")
                continue
            try:
                foreclosure_case = get_foreclosure_case_data(case_url)
            except Exception as e:
                logging.error(f"Error fetching foreclosure case {case_url}: {e}")
                continue
            
            forclosure_case_db = ForeclosureCaseSchema(**foreclosure_case.model_dump())
            forclosure_case_db.bundesland_code = land_code
            session.add(forclosure_case_db)

session.commit()

Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure Case already expired!
Forclosure

In [107]:
session.close()
engine.dispose()
for handler in logging.getLogger().handlers:
    handler.flush()
    handler.close()